Get all Wikidata objects and checks if they are connected to an OSM object using an [API](https://osm.wikidata.link/tagged)

* this [Notebook](https://github.com/salgo60/ProjectOutdoorGyms/blob/main/Jupyter/OSM_Wikidata.ipynb)

* API [Wikidata to OpenStreetMap](https://osm.wikidata.link/tagged)
  * eg. [https://osm.wikidata.link/tagged/api/item/Q106708773](https://osm.wikidata.link/tagged/api/item/Q106708773)

* Another tool [osm.wikidata.link](https://osm.wikidata.link/search)
  

TODO: 
* 


In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2023-08-06 23:40:17.359246


In [2]:
import pandas as pd


In [3]:
#
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys,json
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"
 
#
queryWD = """SELECT DISTINCT (REPLACE(STR(?site), ".*Q", "Q") AS ?qid) ?site WHERE {
  #?site wdt:P17 wd:Q34. 
  ?site wdt:P31/wdt:P279* wd:Q567998.
  ?site wdt:P625 ?coordinates.
#  minus {?site wdt:P17/wdt:P30 wd:Q46} #Europe
   minus {
    { ?site wdt:P10689 ?OSMid. }
    UNION
    { ?site wdt:P402 ?OSMrelid. }
    UNION 
    { ?site wdt:P11693 ?OSMnode. }
  }
} limit 150000"""


def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDo = get_sparql_dataframe(endpoint_url, queryWD )
WDo["Source"] = "WD"     
WDo.shape

(3590, 3)

In [4]:
WDo.head()

,qid,site,Source
0,Q106681580,http://www.wikidata.org/entity/Q106681580,WD
1,Q106687320,http://www.wikidata.org/entity/Q106687320,WD
2,Q106687350,http://www.wikidata.org/entity/Q106687350,WD
3,Q106687359,http://www.wikidata.org/entity/Q106687359,WD
4,Q106687365,http://www.wikidata.org/entity/Q106687365,WD


In [5]:
import urllib3, json
from tqdm import tqdm
http = urllib3.PoolManager()

listWDo = []
for WD, row in tqdm(WDo.iterrows(), total=WDo.shape[0]):
    url = "https://osm.wikidata.link/tagged/api/item/" + row["qid"] 
    
    new_item = dict()
    new_item['wikidata'] = row["qid"] 
    try:
        r = http.request('GET', url) 
        data = json.loads(r.data.decode('utf-8'))
    except:
        print (r.status, url)
    #print (r.status)
    try:
        #print(data)
        osmid = data["osm"][0]["id"]            
        osmtype = data["osm"][0]["type"]              
    except:
        #print ("error")
        #print(data)
        osmid =""
        osmtype =""
    new_item['osmid'] = osmid  
    new_item['type'] = osmtype 
    
    listWDo.append(new_item)
print (len(listWDo))

100%|██████████| 3590/3590 [38:05<00:00,  1.57it/s]     

3590


In [6]:
OSMtot = pd.DataFrame(listWDo,
                  columns=['wikidata','type','osmid'])
OSMtot.shape


(3590, 3)

In [7]:
pd.set_option('max_colwidth', 400)
OSMtot.head(10)

,wikidata,type,osmid
0,Q106681580,way,944656445
1,Q106687320,,
2,Q106687350,,
3,Q106687359,node,8841342235
4,Q106687365,,
5,Q106687370,node,8176902490
6,Q106687381,node,5791321539
7,Q106687395,,
8,Q106687473,,
9,Q106687513,node,8792841179


In [8]:
#OSMempty = OSMtot.osmid.notnull()
OSMtot[(OSMtot['osmid']=="")].shape

(2188, 3)

In [9]:
OSMEmpty =OSMtot[(OSMtot['osmid']=="")]

In [10]:
OSMEmpty.shape

(2188, 3)

In [11]:
OSMEmpty.to_csv("WD - OSM church 1 missing.csv")

OSMEmpty.head()

,wikidata,type,osmid
1,Q106687320,,
2,Q106687350,,
4,Q106687365,,
7,Q106687395,,
8,Q106687473,,


In [12]:
OSMConnected=OSMtot[(OSMtot['osmid']!="")]
OSMConnected.to_csv("WD - OSM_church1.csv")
OSMConnected.head()

,wikidata,type,osmid
0,Q106681580,way,944656445
3,Q106687359,node,8841342235
5,Q106687370,node,8176902490
6,Q106687381,node,5791321539
9,Q106687513,node,8792841179


In [13]:
print("*", start_time.strftime("%Y%m%d"),"WD objects", WDo.shape[0], "ej OSM kopplade",OSMEmpty.shape[0]) 


* 20230806 WD objects 3590 ej OSM kopplade 2188


Generate Markdown table eg.
| 20210526     | 2802 | 2050 |1147 | 254 | 213| 84|


In [14]:
print("|",start_time.strftime("%Y%m%d"),"|", \
      WDo.shape[0],"|",OSMEmpty.wikidata.nunique(),"|")


| 20230806 | 3590 | 2188 |


In [15]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2023-08-07 00:18:25.034920
Time elapsed (hh:mm:ss.ms) 0:38:07.675868
